# Assignment-2

1. Implement Backpropagation algorithm to train an ANN of configuration __2x2x1__ to achieve __XOR__ function.

__Answer__: Neural network is defined and while training backpropagation algorithm is used for adjusting the weights so that the weights are adjusted making the neural network best fit to data as possible.

The XOR truth table looks like:

| Input      | Output |
| ---------- | ------ |
| 0    ,   0 |   0    |
| 1    ,   0 |   1    |
| 0    ,   1 |   1    |
| 1    ,   1 |   0    |


In [1]:
import numpy as np

def sigmoid( net_output ):
        output = 1 / ( 1 + np.exp(-net_output))
        return output
    
class Layer:
    def __init__(self, neurons, no_of_wghts, layer_no):
        print(f"Layer {layer_no+1} has {neurons} neurons and each neuron has {no_of_wghts} weights connected to it.")
        #create weights array of size (neurons, weights+1)
        self.wght = np.random.normal(size=(neurons, no_of_wghts+1))
        self.layer_no = layer_no+1
        self.grad = np.empty(shape=(neurons, no_of_wghts+1))
        self.delta = np.empty(shape=(1, neurons))
        # print(self.wght)
    def activate_h(self, input):
        #adding a single row for bias 
        _, col = input.shape
        # print(input)
        if self.wght.shape[1] == col:
            self.activation = np.dot(input, self.wght.T)
            # print(self.activation)
            self.n_output = sigmoid(self.activation)
            self.n_output = np.c_[self.n_output, np.ones(shape=(self.n_output.shape[0], 1))]
            return self.n_output
        else:
            raise Exception(f"input {input.shape} and weights {self.wght.shape} shape not valid ")
        
    def activate_o(self, input):
        #adding a single row for bias 
        _, col = input.shape
        # print(input)
        if self.wght.shape[1] == col:
            self.activation = np.dot(input,self.wght.T )
            # print(self.activation)
            self.n_output = sigmoid(self.activation)
            return self.n_output
        else:
            raise Exception(f"input {input.shape} and weights {self.wght.shape} shape not valid ")
        


In [2]:
class Input(Layer):
    def __init__(self, neurons, layer_no):
        self.neurons = neurons
        self.layer_no = layer_no+1
    def activate_h(self, input):
        self.n_output = input

In [13]:

class Network:
    def __init__(self, *args) -> None:
        self.layers = []
        self.total_layers = len(args)
        # print(self.total_layers)
        self.layers.append(Input(args[0],0))
        for l in range(1, self.total_layers):
            self.layers.append(Layer(args[l], args[l-1], l))
        # print(self.layers)
    def forward(self, input):
        self.input = input
        input = np.c_[input, np.ones(shape=(input.shape[0],1))]
        self.layers[0].activate_h(input)
        for layer_no in range(1, self.total_layers-1):
            input = self.layers[layer_no].activate_h(input)
            # print(input, self.layers[layer_no].layer_no)
        self.output = self.layers[-1].activate_o(input)
        # print(self.output, self.layers[-1].layer_no)
        
        
    def backward(self, target, learning_rate=10):
        diff = target - self.output
        derivative = self.output * ( 1 - self.output)
        delta = diff * derivative
        # print(delta)
        # print("output=")
        # print(delta.T  @ self.layers[-2].n_output )
        
        # print("weight=")
        # print(self.layers[-1].wght)
        # print(len(delta[:])) 
        #output layer weight update
        for d in range(len(self.output)):
            # print("==================================================")
            # print("target")
            # print(target[d])
            # print("output")
            # print(self.output)
            # print(self.output[d])
            diff = target[d] - self.output[d]
            # print("diff")
            # print(diff)
            derivative = self.output[d] * (1 - self.output[d])
            # print("deriv")
            # print(derivative)
            delta = diff * derivative
            # print("delta=")
            # print(delta)
            # print("grad_0=")
            grad  = np.dot(delta.T[:,np.newaxis], self.layers[-2].n_output[d][np.newaxis,:])
            # print(grad)
            self.layers[-1].grad = grad
            self.layers[-1].delta = delta
            # print(self.layers[-2].n_output[d][np.newaxis, :] )
            #finding the gradients for weight update
            for l in range(self.total_layers-1, 1, -1):
                # print("weights")
                # print(self.layers[l].wght[:, :-1])
                delta_h = np.dot(self.layers[l].delta, self.layers[l].wght[:, :-1]) # don't need the bias for the previous layers' weight update
                deriv_h = (self.layers[l-1].n_output[d] * ( 1- self.layers[l-1].n_output[d]))[np.newaxis, :-1]
                delta_h = delta_h * deriv_h
                # print(delta_h.T, self.layers[l-2].n_output[d][np.newaxis, :]) 
                grad_h = np.dot(delta_h.T, self.layers[l-2].n_output[d][np.newaxis, :]) 
                self.layers[l-1].grad = grad_h
                self.layers[l-1].delta = delta_h
                # print(self.layers[l].layer_no)
                # print("delta_h=")
                # print(delta_h)
                # print("grad_h")
                # print(grad_h)
                # print("hidden=", self.layers[l-1].layer_no)
                # print(self.layers[l-1].n_output[d] * ( 1- self.layers[l-1].n_output[d]))
                # print(self.layers[l].delta.shape,  (self.layers[l-1].n_output[d] * ( 1- self.layers[l-1].n_output[d]))[np.newaxis, :].shape)
                # grad  = np.dot(self.layers[].T[:,np.newaxis], self.layers[self.total_layers-(l+1)].n_output[d][np.newaxis,:])
            
            # print("grad=")
            # print(grad)
        #     update = learning_rate * grad
        #     # print("update=", d)
        #     # print(update)
        #     #iterate through each examples' update array
        #     for u in update:
        #         self.layers[-1].wght[d] += u
        #     print("weightchanged=")
        #     print(self.layers[-1].wght)
                
            #weight update
            for layer in  range(1, self.total_layers):
                # print("layer no", self.layers[layer].layer_no, layer)
                # print("before update", self.layers[layer].wght)
                self.layers[layer].wght = self.layers[layer].wght + (learning_rate * self.layers[layer].grad)
                # print("after update", self.layers[layer].wght)
            self.forward(self.input)     
            
    def train(self, input, output, learning_rate=1, epochs=1000):
        for e in range(1, epochs+1):
            self.forward(input)
            self.backward(output, learning_rate=learning_rate)             
            print(f"{e} epochs completed.", end="\r")
        print("",end='\n')
    
    def test(self, input):
        self.forward(input)
        print(self.output)

In [16]:
np.random.seed(42)

net = Network(2, 2, 1)

# test = Layer(2, 2, 1)
input = np.array([[1, 0],
                  [0, 1],
                  [0, 0],
                  [1, 1]])
output = np.array([ [1],
                    [1],
                    [0],
                    [0] ])
net.train(input, output)
net.test(np.array([[0, 0]]))


Layer 2 has 2 neurons and each neuron has 2 weights connected to it.
Layer 3 has 1 neurons and each neuron has 2 weights connected to it.
1000 epochs completed.
[[0.0559636]]
